In [1]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import os

2025-05-10 17:53:04.684474: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746899584.924274      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746899584.979523      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import torch
from transformers import BertForQuestionAnswering, BertTokenizer
import pandas as pd
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

# Initialize model and tokenizer
model_name = "deepset/deberta-v3-large-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Move model to CUDA if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(f"Using device: {device}")

def find_answer_text(question, text):
    try:
        # Tokenize input and move to same device as model
        inputs = tokenizer.encode_plus(question, text, return_tensors='pt', truncation=True, max_length=512)
        input_ids = inputs['input_ids'].to(device)
        token_type_ids = inputs['token_type_ids'].to(device)
        
        # Get model predictions
        with torch.no_grad():
            output = model(input_ids, token_type_ids=token_type_ids)
        
        # Get answer span
        answer_start = torch.argmax(output.start_logits)
        answer_end = torch.argmax(output.end_logits) + 1
        
        # Convert tokens to answer string
        answer = tokenizer.convert_tokens_to_string(
            tokenizer.convert_ids_to_tokens(input_ids[0][answer_start:answer_end])
        )
        
        return answer.capitalize()
    
    except Exception as e:
        print(f"Error processing: {str(e)}")
        return None

tokenizer_config.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.65M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

Using device: cuda


In [5]:
question = 'What are the problems the authors are solving?'
question = 'What solution is presented?'
question = 'What problem is presented?'
question = 'What research problem is adressed?'
# question = 'What are the limitations of the solution?'
text = '''KOMPSAT-3 is a high performance remote sensing satellite, which provides 0.7 m GSD
panchromatic image and 2.8 m GSD multi-spectral image data for various applications.
KOMPSAT-3 was launched into a sun synchronous low Earth orbit on the 18th of May, 2012
and the life time of more than 7 years is expected.'''
text2 = '''Forecasting high-impact research topics
via machine learning on evolving knowledge graphs
Xuemei Gu1, ∗ and Mario Krenn1, †
1Max Planck Institute for the Science of Light, Staudtstrasse 2, 91058 Erlangen, Germany
The exponential growth in scientific publications poses a severe challenge for human researchers.
It forces attention to more narrow sub-fields, which makes it challenging to discover new impactful
research ideas and collaborations outside one’s own field. While there are ways to predict a scientific
paper’s future citation counts, they need the research to be finished and the paper written, usually
assessing impact long after the idea was conceived. Here we show how to predict the impact of
onsets of ideas that have never been published by researchers. For that, we developed a large
evolving knowledge graph built from more than 21 million scientific papers. It combines a semantic
network created from the content of the papers and an impact network created from the historic
citations of papers. Using machine learning, we can predict the dynamic of the evolving network
into the future with high accuracy (AUC values beyond 0.9 for most experiments), and thereby the
impact of new research directions. We envision that the ability to predict the impact of new ideas
will be a crucial component of future artificial muses that can inspire new impactful and interesting
scientific ideas.
'''

answer = find_answer_text(question,text)
answer

'[cls]'

In [6]:
import pandas as pd

# Load the entire JSON file into a DataFrame in one go
df = pd.read_json('/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json', lines=True)

# Check the DataFrame structure
print(f"Shape: {df.shape}")  # (rows, columns)
print(df.head())

Shape: (2725401, 14)
          id           submitter  \
0  0704.0001      Pavel Nadolsky   
1  0704.0002        Louis Theran   
2  0704.0003         Hongjun Pan   
3  0704.0004        David Callan   
4  0704.0005  Alberto Torchinsky   

                                             authors  \
0  C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...   
1                    Ileana Streinu and Louis Theran   
2                                        Hongjun Pan   
3                                       David Callan   
4           Wael Abu-Shammala and Alberto Torchinsky   

                                               title  \
0  Calculation of prompt diphoton production cros...   
1           Sparsity-certifying Graph Decompositions   
2  The evolution of the Earth-Moon system based o...   
3  A determinant of Stirling cycle numbers counts...   
4  From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...   

                                  comments  \
0  37 pages, 15 figures; published version

In [13]:
df.head(2)

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,0704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,A fully differential calculation in perturba...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,..."
1,0704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,None,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...","[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-12-13,"[[Streinu, Ileana, ], [Theran, Louis, ]]"


In [8]:
import pandas as pd
from transformers import pipeline
import time
results = pd.DataFrame(columns=['id', 'solution', 'problem'])
results.head()

,id,solution,problem


In [9]:
# Process each abstract (adjust batch size based on your resources)
batch_size = 100  # Process in batches to manage memory
t1 = time.time()
for i in range(0, len(df), batch_size):
    batch = df.iloc[i:i+batch_size]
    
    # Get answers for both questions
    solution = batch['abstract'].apply(lambda x: find_answer_text('What solution and idea they presented?', x))
    problem = batch['abstract'].apply(lambda x: find_answer_text('What research problem is adressed?', x))
    
    # Store results
    batch_results = pd.DataFrame({
        'id': batch['id'],
        'solution': solution,
        'problem': problem
    })
    
    results = pd.concat([results, batch_results], ignore_index=True)
    print(results.shape)
    # Progress tracking
    if i % 100 == 0:
        print(f"Processed {i}/{len(df)} abstracts")
        print("passed time:")
        print(time.time()-t1)

# Save results
results.to_csv('arxiv_qa_results.csv', index=False)
print("Processing complete!")
print(results.head())

(100, 3)
Processed 0/2725401 abstracts
passed time:
14.232053756713867


KeyboardInterrupt: 

In [11]:
results[:30]

,id,solution,problem
0,0704.0001,Enhanced sensitivity to the signal can be obta...,Production of massive photon pairs at hadron c...
1,0704.0002,A new decomposition that certifies sparsity,Tree decompositions of graphs
2,0704.0003,The result suggests that the tidal friction ma...,Tidal friction may not be the primary cause fo...
3,0704.0004,The proof involves a bijection from these auto...,A determinant of stirling cycle numbers counts...
4,0704.0005,"How to compute the $\lambda_{\alpha}$ norm, $\...",Compute the $\lambda_{\alpha}$ norm
5,0704.0006,Both definitions give similar results,Two-particle wave function of paired atoms in ...
6,0704.0007,Polymer description arises as an appropriate l...,The relation between the ordinary schroedinger...
7,0704.0008,Illustrating the effects of plastic work on th...,[cls]
8,0704.0009,A set of criteria for isolating bona fide youn...,"Isolating bona fide young stellar objects, yso..."
9,0704.0010,New proofs of some known results are given,[cls]
